In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('data/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Load and check data

In [ ]:
train = pd.read_csv("train.csv")
test_x = pd.read_csv("test_x.csv")
train

# Preprocess text
You may consider applying some preprocessing steps first, such as:
- URL data removal;
- Turn to lowercase;
- Remove hashtags start with @;
- Remove special characters (e.g., =!#%@$^&*)
- Lemmatization (convert words to base form): https://www.nltk.org/howto/stem.html
- ...

In [ ]:
import re

#HC Lemmatize
from nltk.stem import *
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')


# Example: URL removal
def remove_url(data):
    re1 = r"http://\S+|www\.\S+"
    re2 = r"https://\S+|www\.\S+"
    url_clean= re.compile("(%s|%s)" % (re1, re2))
    data=url_clean.sub(r'',data)
    return data

def to_lowercase(data):
    return data.lower()

def remove_hashtag(data):
    return data.replace("@", "")

def remove_special(data):
    return re.sub('\W+',' ', data )

def lemmatize(data):
    tokenized_words=word_tokenize(data)
    stemmer=LancasterStemmer()
    tokenized_sentence = []
    for word in tokenized_words:
        tokenized_sentence.append(stemmer.stem(word))

    return " ".join(tokenized_sentence)

print('before removal: ', train.iloc[5325]['text'])

train['text'] = train['text'].apply(remove_url)
test_x['text'] = test_x['text'].apply(remove_url)

train['text'] = train['text'].apply(to_lowercase)
test_x['text'] = test_x['text'].apply(to_lowercase)

train['text'] = train['text'].apply(remove_hashtag)
test_x['text'] = test_x['text'].apply(remove_hashtag)


train['text'] = train['text'].apply(remove_special)
test_x['text'] = test_x['text'].apply(remove_special)

train['text'] = train['text'].apply(lemmatize)
test_x['text'] = test_x['text'].apply(lemmatize)

print('after removal: ', train.iloc[5325]['text'])

# Classification based on bag-of-word text features


Generating hand-crafted bag-of-word feature vectors:
- Assign a fixed index to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices). Each word essentially is an one-hot embedding (dimension=vocab size);
- For each document #i, count the number of occurrences of each word w and store it in X[i, j] as the value of feature #j where j is the index of word w in the dictionary. Each document is a summation of one-hot embedding of all words.
- You could filter out words that are rarely or too frequently appeared in documents.

sklearn tutorial: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [ ]:
# This package generates bag of words feature for each text
from sklearn.feature_extraction.text import CountVectorizer

# doc for this function: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
#vectorizer = CountVectorizer(stop_words='english', max_df=1.0, min_df=1, max_features=10000)
vectorizer = CountVectorizer(stop_words='english', max_df=1.0, min_df=0.001, max_features=10000)#HC
#HC max_df = 0.90 "ignore terms that appear in more than 90% of the documents".
#HC min_df = 0.001 "ignore terms that appear in less than 0.1% of the documents".

# use training data to construct vocabulary and turn training text into BOW vectors
train_bow = vectorizer.fit_transform(train['text'])
vocab = vectorizer.vocabulary_.copy()
# You can change the vocab size by tuning "max_df", "min_df" or "max_features" in "CountVectorizer"
print('vocab size: ', len(vocab))
print (train_bow.shape) # num_train_text * vocab_size

# !!! YOUR TASK 2 (1 point): transform test text into BOW vectors using vectorizer
test_bow = vectorizer.transform(test_x['text'])                                      #ME HILLMER
print (test_bow.shape) # num_train_text * vocab_size


In [ ]:
# Use basic ML models on BOW features
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

# Split given training data into training and validation set
X_train,X_val,y_train,y_val = train_test_split(train_bow, train['target'], test_size=0.3, random_state=11)

LR = RandomForestClassifier()
LR.fit(X_train.toarray(),y_train)
print('training recall: ', recall_score(y_train, LR.predict(X_train.toarray())))
print('validation recall: ', recall_score(y_val, LR.predict(X_val.toarray())))


# Classification based on word2vec embedding features

word2vec:
- Skip-grams or CBOW
- low-dimensional feature vectors with semantic information preserved

Before deep learning pervades, gensim is a popular text mining package. Here is a tutorial of gensim's word2vec: https://radimrehurek.com/gensim/models/word2vec.html

Now with deep learning, pytorch and tensorflow all has their way to realize word2vec. Tutorial: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

In [ ]:
# train word embeddings using gensim library

from gensim.models import Word2Vec

train_docs = [train['text'].iloc[i].split(' ') for i in range(len(train['text']))] # gensim only takes list of word list as `sentences`
test_x_docs = [test_x['text'].iloc[i].split(' ') for i in range(len(test_x['text']))] # HC

w2v_model = Word2Vec(sentences=train_docs, vector_size=100, window=5, negative=20, min_count=2, workers=1)#original
w2v_model.train(train_docs, total_examples=len(train_docs), epochs=10)

In [ ]:
# check the learned word embeddings

# vocabulary
w2v_vocav = w2v_model.wv.index_to_key
print('example words in vocab: ', [w2v_vocav[i] for i in range(200)])

example_word = 'bomb'
word_vec = w2v_model.wv[example_word]
w2v_model.wv.most_similar(positive=word_vec, topn=10)                             #HC

In [ ]:
train_docs = [train['text'].iloc[i].split(' ') for i in range(len(train['text']))] # gensim only takes list of word list as `sentences`
test_x_docs = [test_x['text'].iloc[i].split(' ') for i in range(len(test_x['text']))] # HC

w2v_model = Word2Vec(sentences=train_docs, vector_size=1556, window=5, negative=10, min_count=2, workers=1)
w2v_model.train(train_docs, total_examples=len(train_docs), epochs=10)

# vocabulary
w2v_vocav = w2v_model.wv.index_to_key
print('example words in vocab: ', [w2v_vocav[i] for i in range(200)])

# example embedding
example_word = 'bomb'
word_vec = w2v_model.wv[example_word]
w2v_model.wv.most_similar(positive=word_vec, topn=10)                                                   

In [ ]:
# Use basic ML models on word embeddings

def doc2vec(docs, w2v_model):
    w2v = []
    for doc in docs:
        word_embs = [w2v_model.wv[w] for w in doc if w in w2v_model.wv] # get a list of word embeddings

        doc_emb = np.mean(np.array(word_embs), axis=0)

        # filling zeros for empty doc
        if len(word_embs) == 0:
            doc_emb = [0 for i in range(w2v_model.vector_size)]

        w2v.append(doc_emb)

    return np.array(w2v)

# transform training tweets into embeddings
train_w2v = doc2vec(train_docs, w2v_model)
print('training word2vec shape: ', train_w2v.shape)

test_w2v = doc2vec(test_x_docs, w2v_model)
print('test word2vec shape: ', test_w2v.shape)

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_train_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_docs)]
tagged_test_x = [TaggedDocument(doc, [i]) for i, doc in enumerate(test_x)]

d2v_model = Doc2Vec(tagged_train_docs, vector_size=1556, window=5, negative=10, min_count=2, workers=20)


def doc2vec(docs, d2v_model):
    w2v = []
    for doc in docs:
        w2v.append(d2v_model.infer_vector(doc))
    return np.array(w2v)

train_d2v = doc2vec(train_docs, d2v_model)
print('test doc2vec shape: ', train_d2v.shape)

test_d2v = doc2vec(test_x_docs, d2v_model)
print('test doc2vec shape: ', test_d2v.shape)

In [ ]:
# Use basic ML models on word2vec features
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

# Split given training data into training and validation set
X_train,X_val,y_train,y_val = train_test_split(train_w2v, train['target'], test_size=0.3, random_state=11) # Using Word2Vec


LR = RandomForestClassifier()
LR.fit(X_train,y_train)
print('training recall: ', recall_score(y_train, LR.predict(X_train)))
print('validation recall: ', recall_score(y_val, LR.predict(X_val)))

# Optional (+1 bonus pt): Classification directly using RNN model
RNN model can take in word, turn them into embedding and make predictions

Torch tutorial: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

In [ ]:
#Prerequisites
!pip install 'portalocker>=2.0.0'

In [ ]:
train.nunique()


In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_label(df, column_name):
  temp_df = df
  le = LabelEncoder()
  temp_df['label'] =  le.fit_transform(temp_df[column_name])+1
  temp_df = temp_df.drop([column_name], axis=1)
  temp_df = temp_df[['label'] + [col for col in temp_df.columns if col != 'label']]
  return temp_df

train_rnn = encode_label(train[['text', 'keyword']], 'keyword')
train_rnn.nunique()

train_rnn

#train_rnn.groupby(['label']).count()


In [ ]:
# Preprocess text via tokenizer

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
train_iter = train_rnn['text']#HC

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
print('size of vocab: ', len(vocab))

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [ ]:
from torch.utils.data import DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


train_iter = list(train_rnn.itertuples(index=False))
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [ ]:
# Build a simple RNN model

from torch import nn

# RNN can be a combination of feature extractor (word2vec) and a classifer
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False) # word2vec
        self.fc = nn.Linear(embed_dim, num_class) # classifier
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:

train_iter = list(train_rnn.itertuples(index=False))
num_class = len(set([label for (label, text) in train_iter]))

vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

print(num_class)


In [ ]:
#Define functions to train the RRN model and evaluate results

import time

def trainRNN(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluateRNN(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count


In [ ]:
# Split the dataset and run the model

from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None


train_iter = list(train_rnn.itertuples(index=False))

test_rnn = encode_label(test_x[['text', 'keyword']], 'keyword')
test_iter = list(test_rnn.itertuples(index=False)) #HC

train_dataset = to_map_style_dataset(train_iter) #HC
test_dataset = to_map_style_dataset(test_iter) #HC

num_train = int(len(train_dataset) * 0.95)


split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)


train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)


for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    trainRNN(train_dataloader)
    accu_val = evaluateRNN(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

# Save submission file

In [ ]:
#prediction_y_pred = LR.predict(test_bow.toarray())
prediction_y_pred = LR.predict(test_d2v)
print(prediction_y_pred)

y_pred = pd.DataFrame({
    'id': test_x['id'],
    'target': prediction_y_pred
    }
)

y_pred.to_csv("test_submission-vf-2d2v.csv", index=False)
y_pred.head(3)